In [11]:
%%writefile quants/2_SimulatingTrades.py

import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr

yf.pdr_override()
now = datetime.now()
start_date = now - relativedelta(years=2)
end_date = now - relativedelta(days=1)

stock = input("Enter a stock ticker symbol: ")
df = fdr.DataReader(stock, start_date, end_date) # for last 12 months
df = df.rename(columns={'Close': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low',  'Volume': 'volume', 'Change': 'change'})

# ma=50
# smaString="Sma_"+str(ma)
# df[smaString]=df.iloc[:,4].rolling(window=ma).mean()

emasUsed = [3,5,8,10,12,15,30,35,40,45,50,60]
for x in emasUsed:
    ema = x
    df['EMA_'+str(ema)] = round(df.iloc[:, 0].ewm(span=ema,adjust=False).mean(),20)

df = df.iloc[60:]

# print(df)

pos = 0
percentchange = []
num = 0
for i in df.index:
    emasUsed = [3,5,8,10,12,15,30,35,40,50,60]
    cmin=min(df['EMA_3'][i],df['EMA_5'][i],df['EMA_8'][i],df['EMA_10'][i],df['EMA_12'][i],df['EMA_15'][i])
    cmax=max(df['EMA_30'][i],df['EMA_35'][i],df['EMA_40'][i],df['EMA_45'][i],df['EMA_50'][i],df['EMA_60'][i])

    close = df['close'][i]
    if((cmin>cmax)):
#         print("Red Write Blue")
        if(pos==0):
            bp=close
            pos=1
            print("Buying now at "+str(bp))
    elif(cmin<cmax):
#         print("Blue Write Red")
        if(pos==1):
            pos=0
            sp=close
            print("Selling now at "+str(sp))
            pc=(sp/bp-1)*100
            percentchange.append(pc)
          
    if(num==df['close'].count()-1 and pos ==1):
        pos=0
        sp=close
        print("Selling now at "+str(sp))
        pc=(sp/bp-1)*100
        percentchange.append(pc)
        
    num+=1
# print(percentchange)

gains=0
ng=0
losses=0
nl=0
totalR=1

for i in percentchange:
    if(i>0):
        gains+=i
        ng+=1
    else:
        losses+=i
        nl+=1
    totalR=totalR*((i/100)+1)
totalR=round((totalR-1)*100, 2)

if(ng>0):
    avgGain=gains/ng
    maxR=str(max(percentchange))
else:
    avgGain=0
    maxR='undefined'

if(nl>0):
    avgLoss=losses/nl
    maxL=str(min(percentchange))
    ratio=str(-avgGain/avgLoss)
else:
    avgLoss=0
    maxL='undefined'
    ratio='inf'
    
if(ng>0 or nl>0):
    battingAvg=ng/(ng+nl)
else:
    battingAvg=0

print()
print("Results for "+ stock +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAvg))
print("Gain/Loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR) + "%")
print()

Overwriting quants/2_SimulatingTrades.py


In [9]:
%run quants/2_SimulatingTrades.py

Enter a stock ticker symbol:  GOOGL


Buying now at 1217.8
Selling now at 1419.9
Buying now at 1317.3
Selling now at 1362.5
Buying now at 1442.0
Selling now at 1473.3
Buying now at 1504.9
Selling now at 1480.5
Buying now at 1507.2
Selling now at 1547.2
Buying now at 1555.5
Selling now at 1726.2
Buying now at 1774.0
Selling now at 1726.1
Buying now at 1774.3
Selling now at 1730.9
Buying now at 1784.5
Selling now at 2024.7
Buying now at 2045.8
Selling now at 2200.3
Buying now at 2278.4
Selling now at 2510.4

Results for GOOGL going back to 2019-10-04 00:00:00, Sample size: 11 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 50, 60]
Batting Avg: 0.7272727272727273
Gain/Loss ratio: 3.7137088447876594
Average Gain: 8.377530467735376
Average Loss: -2.2558393287867946
Max Return: 16.5955000821153
Max Loss: -2.700112739571592
Total return over 11 trades: 76.23%



In [1]:
# %load quants/3_StockScreener.py

import FinanceDataReader as fdr
import os
import pandas as pd
import numpy as np
import time
import yfinance as yf
import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr
from pandas import ExcelWriter

dataPath='./quants/data/'
reportPath='./quants/reports/'

# Checking Minervini conditions of top 30% of stocks in given list
rs_stocks = rs_df['Ticker']
for stock in rs_stocks:    
    try:
        df = pd.read_csv(f'{dataPath}{stock}.csv', index_col=0)
        sma = [50, 150, 200]
        for x in sma:
            df["SMA_"+str(x)] = round(df['Close'].rolling(window=x).mean(), 2)
        
        # Storing required values 
        currentClose = df["Close"][-1]
        moving_average_50 = df["SMA_50"][-1]
        moving_average_150 = df["SMA_150"][-1]
        moving_average_200 = df["SMA_200"][-1]
        low_of_52week = round(min(df["Low"][-260:]), 2)
        high_of_52week = round(max(df["High"][-260:]), 2)
        RS_Rating = round(rs_df[rs_df['Ticker']==stock].RS_Rating.tolist()[0])
        
        try:
            moving_average_200_20 = df["SMA_200"][-20]
        except Exception:
            moving_average_200_20 = 0

        # Condition 1: Current Price > 150 SMA and > 200 SMA
        condition_1 = currentClose > moving_average_150 > moving_average_200
        
        # Condition 2: 150 SMA and > 200 SMA
        condition_2 = moving_average_150 > moving_average_200

        # Condition 3: 200 SMA trending up for at least 1 month
        condition_3 = moving_average_200 > moving_average_200_20
        
        # Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
        condition_4 = moving_average_50 > moving_average_150 > moving_average_200
           
        # Condition 5: Current Price > 50 SMA
        condition_5 = currentClose > moving_average_50
           
        # Condition 6: Current Price is at least 30% above 52 week low
        condition_6 = currentClose >= (1.3*low_of_52week)
           
        # Condition 7: Current Price is within 25% of 52 week high
        condition_7 = currentClose >= (.75*high_of_52week)
        
        # If all conditions above are true, add stock to exportList
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7):
            exportList = exportList.append({'Stock': stock, "RS_Rating": RS_Rating ,"50 Day MA": moving_average_50, "150 Day Ma": moving_average_150, "200 Day MA": moving_average_200, "52 Week Low": low_of_52week, "52 week High": high_of_52week}, ignore_index=True)
#             print (stock + " made the Minervini requirements")
    except Exception as e:
        print (e)
        print(f"Could not gather data on {stock}")

exportList = exportList.sort_values(by='RS_Rating', ascending=False)
print('\n', exportList)
writer = ExcelWriter(f'{reportPath}ScreenOutput.xlsx')
exportList.to_excel(writer, "Sheet1")
writer.save()


In [ ]:
%run quants/3_StockScreener.py

In [39]:
# %load quants/4_emailskeleton.py
import os
import smtplib
import imghdr
from email.message import EmailMessage

import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr

EMAIL_ADDRESS = os.environ.get('EMAIL_USER')
EMAIL_PASSWORD = os.environ.get('EMAIL_PASS')